
Author: Ara Alexandrian <br>Date: September 28, 2023

This notebook contains functions for connecting to a Redis server, storing data to Redis, fetching specific events from Redis, converting events to a pandas DataFrame, and clearing the Redis cache.

The functions included are:

connect_to_redis: Connects to a Redis server using the provided host, port, and password.
store_data_to_redis: Stores data from a pandas DataFrame to Redis.
fetch_near_miss_events: Fetches all near-miss events from Redis, with an optional location filter.
events_to_dataframe: Converts a list of events to a pandas DataFrame.
clear_redis_cache: Clears all keys in all databases of the Redis server.
To use these functions, you need to provide the Redis server details, such as host, port, and password. You also need to provide the path to an Excel file containing the data you want to store in Redis.

Please note that you need to have the pandas and redis libraries installed in order to run these functions.



In [5]:
import pandas as pd
import redis

In [6]:
def connect_to_redis(redis_host, redis_port, redis_password):
    try:
        # Connect to Redis server
        r = redis.StrictRedis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)
        return r
    except redis.exceptions.ConnectionError as e:
        print(f"Failed to connect to Redis server: {e}")
        return None

def store_data_to_redis(r, df):
    try:
        for index, row in df.iterrows():
            # Convert the row to a dictionary and store it in Redis
            # Use 'Event Number' instead of 'EventID'
            event_number = row['Event Number']
            event_data = row.to_dict()
            r.hset(f"event:{event_number}", mapping=event_data)
    except Exception as e:
        print(f"Failed to store data to Redis: {e}")



def fetch_near_miss_events(r, location_filter=None):
    near_miss_events = []
    cursor = 0
    while True:
        cursor, keys = r.scan(cursor=cursor, match='event:*', count=100)
        for key in keys:
            event_data = r.hgetall(key)
            if (not location_filter or event_data.get('Location_Sub') == location_filter) and event_data.get('Classification') == 'Near-miss':
                near_miss_events.append(event_data)
        if cursor == 0:
            break  # Exit the loop when the cursor is 0
    return near_miss_events


def events_to_dataframe(near_miss_events):
    # Convert the list of dictionaries to a pandas DataFrame
    events_df = pd.DataFrame(near_miss_events)
    return events_df

def clear_redis_cache(redis_host, redis_port, redis_password):
    try:
        # Connect to Redis server
        r = redis.StrictRedis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)
        
        # Clear all keys in all databases
        r.flushall()
        print("Redis cache cleared successfully.")
    except redis.exceptions.ConnectionError as e:
        print(f"Failed to connect to Redis server: {e}")

In [7]:
# Load the data from Excel file
excel_file_path = r'extracted.xlsx'  # Change this to your actual file path
df = pd.read_excel(excel_file_path)

# Assuming the Redis server is on localhost and the default port, with no password
# Change these to your actual Redis server details
redis_host = "192.168.1.4"
redis_port = 6379
redis_password = ""


# Clear Redis cache uncomment to use
clear_redis_cache(redis_host, redis_port, redis_password)

# Fetch near miss events from Redis and store it in a DataFrame
r = connect_to_redis(redis_host, redis_port, redis_password)
if r is not None:
    # Store the data from the spreadsheet to Redis
    store_data_to_redis(r, df)


Redis cache cleared successfully.


In [8]:
    # Fetch near miss events from Redis
    near_miss_events = fetch_near_miss_events(r, '')
    events_df = events_to_dataframe(near_miss_events)

    # Print the resulting DataFrame
    print(events_df)


                                         Location_Sub    Date_Time_Submitted  \
0    HSROC, Hazelrig-Salter Radiation Oncology Center  4/12/2022 12:24:40 PM   
1    HSROC, Hazelrig-Salter Radiation Oncology Center    7/5/2023 9:02:40 AM   
2            TKC-AR, The Kirklin Clinic at Acton Road  9/11/2023 11:33:47 AM   
3    HSROC, Hazelrig-Salter Radiation Oncology Center   5/5/2023 10:23:47 AM   
4                                   UAB Proton Center   9/16/2022 9:19:00 AM   
..                                                ...                    ...   
102          TKC-AR, The Kirklin Clinic at Acton Road   2/23/2022 3:33:36 PM   
103                                 UAB Proton Center   2/18/2022 7:14:45 PM   
104                                 UAB Proton Center   8/30/2023 9:51:46 AM   
105  HSROC, Hazelrig-Salter Radiation Oncology Center    8/9/2023 4:57:49 PM   
106  HSROC, Hazelrig-Salter Radiation Oncology Center  11/19/2021 6:03:34 PM   

    Event Number Classification  \
0   